## Baseline GCN testing
Notebook to create and evaluate GCN against EBC on predicting number of passing bicyclists in *copenhagen?*
- Preprocess EBC for graph DONE
- Assign Metrics from data
- Create Torch Graph
- Evaluate against SOTA

In [ ]:
import torch
from torch_geometric.data import Data
import torch_geometric as tg
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import HeatMap
from shapely.geometry import Point, LineString, Polygon
import shapely
import momepy as mp 
import esda
import seaborn as sns
sns.set_theme()

lat, lon = 55.6867243, 12.5700724

def get_city_graph(lat, lon, dist, polygonize=False, plot = False):
    g = ox.graph_from_point((lat, lon), dist=dist, network_type='bike', simplify=True, retain_all=False)
    gdf = mp.nx_to_gdf(g)
    edges = gdf[1].to_crs('EPSG:3857')
    if polygonize:
        linestrings = edges.geometry # our geopandas.GeoSeries of linestrings representing street network
        collection = shapely.GeometryCollection(linestrings.array)  # combine to a single object
        noded = shapely.node(collection)  # add missing nodes
        polygonized = shapely.polygonize(noded.geoms)  # polygonize based on an array of noded parts
        polygons = gpd.GeoSeries(polygonized.geoms)  # create a GeoSeries from parts
        return g, edges, polygons
    if plot:
        ### plot the graph
        fig, ax = ox.plot_graph(g, node_size=0, edge_linewidth=0.5, show=False, close=False)
        edges.plot(ax=ax, linewidth=1, edgecolor='black')
        plt.show()
    return g, edges

g, edges = get_city_graph(lat, lon, 10000)

### plot the graph
fig, ax = ox.plot_graph(g, node_size=0, edge_linewidth=0.5, show=False, close=False)
edges.plot(ax=ax, linewidth=1, edgecolor='black')

import nx_parallel as nxp
import networkx as nx

# enabling networkx's config for nx-parallel
nx.config.backends.parallel.active = True

# setting `n_jobs` (by default, `n_jobs=None`)
nx.config.backends.parallel.n_jobs = 4

### carry weights over to line graph
H = nx.line_graph(g)
H.add_nodes_from((node, g.edges[node]) for node in H)   

for s, t, v in H.edges:
    H.edges[(s, t, v)]['weight'] = g.edges[s]['length'] + g.edges[t]['length']

ebc = dict(nxp.all_pairs_dijkstra_path(H, weight='weight', cutoff=1000))


### EBC Calculation

In [2]:
def calc_bc(shortest_paths, graph):
    bc = {i : 0 for i in graph.nodes}
    for node in (graph.nodes):
        for other_node in shortest_paths[node].keys():
            path = shortest_paths[node][other_node]
            for node_visited in path:
                bc[node_visited] += 1
    for node in bc.keys():
        bc[node] /= len(graph.nodes)
    return bc
bc = calc_bc(ebc, H)
bc = {k: v for k, v in sorted(bc.items(), key=lambda item: item[1], reverse=True)}


### color edges in g by bc
bc2 = {}
for x, y, z in bc:
    bc2[(x, y)] = bc[(x, y, z)]

for (s, t), value in bc2.items():
    for i in range(len(g[s][t])):
        g[s][t][i]['bc'] = value


In [3]:
g_copy = g.copy()
for node in list(g_copy.nodes(data=True)):
    for key in list(node[1].keys()):
        del node[1][key]


### Counter data assignment

# TODO

In [27]:
def counter_assigner(graph, station_list):
    ### copy from research project
    pass


### Creating Torch Graph from **edgelist**

To be used as we convert graphs with calculated betweenness centralities and run our GCN over them. <br>
**TODO: Functionize**

In [ ]:
edge_list = []
for s, t, v in list(g_copy.edges(data=True)):
    edge_list.append(((s, t), v['bc']))

# Step 1: Create node mapping (string -> integer)
node_to_idx = {}
for (src, tgt), _ in edge_list:
    if src not in node_to_idx:
        node_to_idx[src] = len(node_to_idx)
    if tgt not in node_to_idx:
        node_to_idx[tgt] = len(node_to_idx)

# Step 2: Extract edge index and features
edge_index = []
edge_attr = []

for (src, tgt), feature in edge_list:
    edge_index.append([node_to_idx[src], node_to_idx[tgt]])
    edge_attr.append(feature)

# Convert to torch tensors
edge_index = torch.tensor(edge_index, dtype=torch.long).t()  # Shape: [2, num_edges]
edge_attr = torch.tensor(edge_attr, dtype=torch.float)  # Shape: [num_edges, feature_dim]

# Create graph object
graph = Data(edge_index=edge_index, edge_attr=edge_attr)

# Print output
print(graph)
